In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [9]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# train_transforms = transforms.Compose([
#     transforms.RandomResizedCrop(224),  # Randomly crop the image and resize it to 224x224
#     transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
#     transforms.RandomRotation(10),      # Randomly rotate the image by a maximum of 10 degrees
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
#     transforms.ToTensor(),              # Convert the image to a PyTorch tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image using ImageNet mean and standard deviation
# ])

In [10]:
torch.manual_seed(0)

train_dataset = torchvision.datasets.ImageFolder(root='train', transform=transform)

batch_size = 128

# train_size = int(0.9 * len(train_dataset))
# val_size = len(train_dataset) - train_size
# trainset, valset = random_split(train_dataset, [train_size, val_size])

test_dataset = torchvision.datasets.ImageFolder(root='test', transform=transform)
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# valloader = DataLoader(valset, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
resnet = models.resnet152(weights=True)
# Freeze convolutional layers
# Get the total number of layers in the model

# Freeze all parameters initially
for param in resnet.parameters():
    param.requires_grad = False

num_classes = 8
# Modify fully connected layers
num_ftrs = resnet.fc.in_features
print(num_ftrs)

#resnet.avgpool = nn.AdaptiveAvgPool2d((1, 1))

resnet.fc = nn.Sequential(
 #   nn.Flatten(),                               # Flatten the output of the previous layer
    nn.Linear(num_ftrs, 128),                   # Adjust the size of the first fully connected layer
    nn.ReLU(),        
    nn.BatchNorm1d(128),
    nn.Dropout(0.5),                            # Dropout layer for regularization
    nn.Linear(128, num_classes),                # Output layer with the number of classes                           # Softmax activation to convert raw scores into probabilities
)

2048


In [12]:
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=4, min_lr = 1e-7, verbose=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)
print(device)

cuda:0


In [13]:
epochs = 10

for epoch in range(epochs):
    # Training phase
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()

        outputs = resnet(inputs)
        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Validation phase
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = resnet(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(testloader)
    scheduler.step(val_loss)
    val_accuracy = 100 * correct / total
    print('Epoch %d, validation loss: %.3f, validation accuracy: %.2f%%' %
          (epoch + 1, val_loss, val_accuracy), "Current lr:", optimizer.param_groups[0]['lr'])

print('Finished Training')


Epoch 1, validation loss: 1.341, validation accuracy: 63.33% Current lr: 0.001
Epoch 2, validation loss: 1.213, validation accuracy: 67.50% Current lr: 0.001
Epoch 3, validation loss: 1.393, validation accuracy: 64.17% Current lr: 0.001
Epoch 4, validation loss: 1.352, validation accuracy: 64.17% Current lr: 0.001
Epoch 5, validation loss: 1.519, validation accuracy: 65.83% Current lr: 0.001
Epoch 6, validation loss: 1.527, validation accuracy: 65.83% Current lr: 0.001
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
Epoch 7, validation loss: 1.509, validation accuracy: 68.33% Current lr: 0.0001
Epoch 8, validation loss: 1.549, validation accuracy: 65.83% Current lr: 0.0001
Epoch 9, validation loss: 1.480, validation accuracy: 67.50% Current lr: 0.0001
Epoch 10, validation loss: 1.606, validation accuracy: 68.33% Current lr: 0.0001
Finished Training


In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy of the network on the %d test images: %.2f %%' % (total, accuracy))

Accuracy of the network on the 120 test images: 68.33 %
